In [21]:
# 最高ウェイト表現のウェイトを最高ウェイトとの差で表す関数。
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [22]:
# Parabolic subgroupを表すクラス
class Parabolic_subgroup:
    def __init__(self, G, L, crossed_out_nodes) -> None:
        self.G = G
        self.L = L
        self.crossed_out_nodes = crossed_out_nodes
        self.R_G = WeylCharacterRing(self.G)
        self.R_L = WeylCharacterRing(self.L)

    def __repr__(self) -> str:
        return f'parabolic subgroup of {self.G} with crossed out nodes {self.crossed_out_nodes}'

    def dynkin_diagram(self):
        return self.G.marked_nodes(self.crossed_out_nodes).dynkin_diagram()

    def ambient_space(self):
        return self.R_G.space()
    
    def ambient_space_dimension(self):
        return self.R_G.space().dimension()

    # Sagemathのライブラリでは1-indexを用いるが、ここでは簡単のためとりあえず0-indexで表す
    def simple_roots(self):
        return [ self.R_G.simple_roots()[i] for i in set(1..self.G.rank()) - set(self.crossed_out_nodes)]

    def positive_roots(self):
        L = self.ambient_space()

        # positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
        roots = [ pr for pr in self.G.root_system().root_lattice().positive_roots()]
        for i in self.crossed_out_nodes:
            roots = [ pr for pr in roots if not (pr - (self.G.root_system().root_lattice().simple_roots())[i]).is_positive_root()]

        # ambient spaceの元に変換
        return [ L(pr) for pr in roots ]

    # 引数の`weight`は基本ウェイトを基底にして表示したもの
    def weight_muliplicities(self, weight) -> dict:
        # GとLのディンキン図の頂点のずれを補正する関数
        def correct_index(index: int) -> int:
            for i in range(len(self.crossed_out_nodes)):
                if index + i < self.crossed_out_nodes[i]:
                    return index + i
            return index + len(self.crossed_out_nodes)

        fws_L = [fw for fw in self.R_L.fundamental_weights()] # conversion from 1-index to 0-index
        weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(self.crossed_out_nodes)] 
        weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(self.L.rank()))

        fws_G = [fw for fw in self.R_G.fundamental_weights()]
        weight_for_G = sum( weight[i] * fws_G[i] for i in range(self.G.rank()))

        mul_set = root_difference_multiplicities(self.R_L, weight_for_L)

        result = {}
        for k, v in mul_set.items():
            w = weight_for_G \
                + sum(k[i - 1] * self.R_G.simple_roots()[correct_index(i)] 
                    for i in (1..self.L.rank()))
            result[w] = v

        return result
    


In [23]:
P = Parabolic_subgroup(CartanType('A3'), CartanType('A2'), [1])
P

parabolic subgroup of ['A', 3] with crossed out nodes [1]

In [24]:
P.dynkin_diagram()

X---O---O
1   2   3   
A3 with node 1 marked

In [25]:
P.ambient_space_dimension()

4

In [26]:
P.simple_roots()

[(0, 1, -1, 0), (0, 0, 1, -1)]

In [27]:
P.positive_roots()

[(0, 1, -1, 0), (0, 0, 1, -1), (0, 1, 0, -1)]

In [28]:
P.weight_muliplicities((1, 2, 0))

{(3, 2, 0, 0): 1,
 (3, 1, 1, 0): 1,
 (3, 0, 2, 0): 1,
 (3, 1, 0, 1): 1,
 (3, 0, 1, 1): 1,
 (3, 0, 0, 2): 1}

In [29]:
# `degree`次部分を取り出す関数
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [30]:
class Flag_variety:
    def __init__(self, parabolic_subgroup) -> None:
        self.parabolic_subgroup = parabolic_subgroup

        # コホモロジー環を含む環
        self.ring = PolynomialRing(QQ, 'x', parabolic_subgroup.ambient_space_dimension()) 
        self.x = self.ring.gens()

        self.tangent_weights = [
            sum( r[l] * self.x[l] for l in range(parabolic_subgroup.ambient_space_dimension())) 
            for r in set(parabolic_subgroup.R_G.positive_roots())-set(parabolic_subgroup.positive_roots())
        ]
        self.dim = len(self.tangent_weights)

    def __repr__(self) -> str:
        return f'a flag_variety associated to {self.parabolic_subgroup}'

    # 次数ごとのchern類
    def chern_classes(self):
        return [ 
            homogeneous_part(prod(1+x for x in self.tangent_weights), i)
            for i in (0..self.dim) 
        ]

    def numerical_integration_by_localization(self, f):
        random_x = [RealField(1000)(random()) for i in range(self.parabolic_subgroup.ambient_space_dimension())]
        orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(self.parabolic_subgroup.G)]
        top_of_f = homogeneous_part(f, self.dim)
        denominator_in_localization = prod(self.tangent_weights)
        return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round() / len(WeylGroup(self.parabolic_subgroup.L))
            

In [31]:
X = Flag_variety(P)
X

a flag_variety associated to parabolic subgroup of ['A', 3] with crossed out nodes [1]

In [32]:
X.tangent_weights

[x0 - x2, x0 - x1, x0 - x3]

In [33]:
X.chern_classes()

[1,
 3*x0 - x1 - x2 - x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 x0^3 - x0^2*x1 - x0^2*x2 + x0*x1*x2 - x0^2*x3 + x0*x1*x3 + x0*x2*x3 - x1*x2*x3]

In [34]:
class Equivariant_vector_bundle:
    def __init__(self, flag_variety, weight) -> None:
        self.flag_variety = flag_variety
        self.weight = weight
        self.weight_muliplicities = flag_variety.parabolic_subgroup.weight_muliplicities(weight)
        self.rank = sum(v for v in self.weight_muliplicities.values())

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight}'

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        cc = prod((1 + class_from_weight(vector(w)))^i for w, i in self.weight_muliplicities.items())

        return [
            homogeneous_part(cc, i)
            for i in (0..self.flag_variety.dim)
        ]

In [35]:
w = (1, 1, 0)
E = Equivariant_vector_bundle(X, w)
E

an equivariant vector bundle on a flag_variety associated to parabolic subgroup of ['A', 3] with crossed out nodes [1] associated to (1, 1, 0)

In [36]:
E.chern_classes()

[1,
 6*x0 + x1 + x2 + x3,
 12*x0^2 + 4*x0*x1 + 4*x0*x2 + x1*x2 + 4*x0*x3 + x1*x3 + x2*x3,
 8*x0^3 + 4*x0^2*x1 + 4*x0^2*x2 + 2*x0*x1*x2 + 4*x0^2*x3 + 2*x0*x1*x3 + 2*x0*x2*x3 + x1*x2*x3]

In [37]:
import re
# `chern_classes`は0次を省いて渡す
def hrr(integral, rank_E, chern_classes_of_E: list, chern_classes_of_M: list) -> int:
    dim_M  = len(chern_classes_of_M)
    len_cE = len(chern_classes_of_E)

    # Define rings where classes lie
    ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cE) ], order=TermOrder('wdeglex', tuple(1..len_cE)))
    ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..dim_M) ], order=TermOrder('wdeglex', tuple(1..dim_M)))

    # Using Singular, calclate universal formula of chern character
    singular.lib('chern.lib')
    r = singular.ring(0, f'(c(1..{len_cE}))', 'dp')
    l = singular.list(f'c(1..{len_cE})')
    ch_str_list = singular.chAll(l, dim_M).sage_structured_str_list()
    chern_character = [ ring_for_ch(rank_E) ] + [ ring_for_ch(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_str_list ]

    # Using Singular, calculate universal formula of Todd classes
    r = singular.ring(0, f'(c(1..{dim_M}))', 'dp')
    l = singular.list(f'c(1..{dim_M})')
    todd_str_list = singular.todd(l).sage_structured_str_list()
    todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub(r'c\(([0-9]+)\)', r'c\1_M', s)) for s in todd_str_list ]
    
    # Return the value of integration
    return integral( 
        sum(
            chern_character[i](chern_classes_of_E)
            * todd_classes[dim_M - i](chern_classes_of_M)
        for i in (0..dim_M))
    )

In [38]:
hrr(
    X.numerical_integration_by_localization,
    E.rank,
    E.chern_classes()[1:],
    X.chern_classes()[1:]
)

20

In [39]:
P.ambient_space().weyl_dimension(sum(w[i - 1] * P.R_G.fundamental_weights()[i] for i in (1..P.G.rank())))

20